In [1]:
import varseek as vk

vk.ref(
    variants="cosmic_cmc",
    sequences="cdna",
    out="trash_temp5/vk_ref_files",
    w=47,
    k=51,
    dlist_reference_source="grch37",
    dlist_reference_ensembl_release=93,
    download=True,
)

23:05:41 - INFO - Using COSMIC email from COSMIC_EMAIL environment variable: jmrich@caltech.edu
23:05:41 - INFO - Using COSMIC password from COSMIC_PASSWORD environment variable
23:05:44 - INFO - Successfully verified COSMIC credentials.
23:05:44 - WARNING - According to COSMIC regulations, please do not share any data that utilizes the COSMIC database. See more here: https://cancer.sanger.ac.uk/cosmic/help/terms
23:05:44 - INFO - Downloading reference files with variants=cosmic_cmc, sequences=cdna


File downloaded successfully as 'trash_temp5/vk_ref_files/vcrs_index.idx'
File downloaded successfully as 'trash_temp5/vk_ref_files/vcrs_t2g.txt'


{'index': 'trash_temp5/vk_ref_files/vcrs_index.idx',
 't2g': 'trash_temp5/vk_ref_files/vcrs_t2g.txt'}

In [1]:
from varseek.utils import vcf_to_dataframe
import pandas as pd

In [2]:
sample_vcf = "/Users/joeyrich/Downloads/sample.vcf"

sample_vcf_df = vcf_to_dataframe(sample_vcf)
sample_vcf_df['CHROM'] = sample_vcf_df['CHROM'].astype(str)

[W::vcf_parse] Contig '19' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig '20' is not defined in the header. (Quick workaround: index the file with tabix.)
[W::vcf_parse] Contig 'X' is not defined in the header. (Quick workaround: index the file with tabix.)


In [15]:
# Explode ALT column
sample_vcf_df["ALT"] = sample_vcf_df["ALT"].str.split(",")  # Convert ALT to list
sample_vcf_df = sample_vcf_df.explode("ALT").reset_index(drop=True)  # Expand into multiple rows

In [19]:
sample_vcf_df["ALT"]

0       C
1       G
2       A
3       C
4       T
5       A
6       G
7       T
8      GA
9     GAC
10      A
11    ATG
Name: ALT, dtype: object

In [3]:
sample_vcf_df

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,NA00001_GT,NA00001_HQ,NA00002_GT,NA00002_HQ,NA00003_GT,NA00003_HQ,INFO_NS,INFO_DP,INFO_AF,INFO_DB,INFO_H2,NA00001_GQ,NA00001_DP,NA00002_GQ,NA00002_DP,NA00003_GQ,NA00003_DP,INFO_AA,INFO_AN,INFO_AC,ALT_ORIGINAL
0,19,111,None,A,C,9.6,"(None,)","(0, 0)","(10, 10)","(0, 0)","(10, 10)","(0, 1)","(3, 3)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,C
1,19,112,None,A,G,10.0,"(None,)","(0, 0)","(10, 10)","(0, 0)","(10, 10)","(0, 1)","(3, 3)",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G
2,20,14370,rs6054257,G,A,29.0,"(PASS,)","(0, 0)","(51, 51)","(1, 0)","(51, 51)","(1, 1)","(None, None)",3.0,14.0,"(0.5,)",True,True,48.0,1.0,48.0,8.0,43.0,5.0,NaN,NaN,NaN,"A,C,T"
3,20,14370,rs6054257,G,C,29.0,"(PASS,)","(0, 0)","(51, 51)","(1, 0)","(51, 51)","(1, 1)","(None, None)",3.0,14.0,"(0.5,)",True,True,48.0,1.0,48.0,8.0,43.0,5.0,NaN,NaN,NaN,"A,C,T"
4,20,14370,rs6054257,G,T,29.0,"(PASS,)","(0, 0)","(51, 51)","(1, 0)","(51, 51)","(1, 1)","(None, None)",3.0,14.0,"(0.5,)",True,True,48.0,1.0,48.0,8.0,43.0,5.0,NaN,NaN,NaN,"A,C,T"
5,20,17330,None,T,A,3.0,"(q10,)","(0, 0)","(58, 50)","(0, 1)","(65, 3)","(0, 0)","(None, None)",3.0,11.0,"(0.017000000923871994,)",NaN,NaN,49.0,3.0,3.0,5.0,41.0,3.0,NaN,NaN,NaN,A
6,20,1110696,rs6040355,A,G,67.0,"(PASS,)","(1, 2)","(23, 27)","(2, 1)","(18, 2)","(2, 2)","(None, None)",2.0,10.0,"(0.3330000042915344, 0.6669999957084656)",True,NaN,21.0,6.0,2.0,0.0,35.0,4.0,T,NaN,NaN,"G,T"
7,20,1110696,rs6040355,A,T,67.0,"(PASS,)","(1, 2)","(23, 27)","(2, 1)","(18, 2)","(2, 2)","(None, None)",2.0,10.0,"(0.3330000042915344, 0.6669999957084656)",True,NaN,21.0,6.0,2.0,0.0,35.0,4.0,T,NaN,NaN,"G,T"
8,20,1234567,microsat1,G,GA,50.0,"(PASS,)","(0, 1)",NaN,"(0, 2)",NaN,"(1, 1)",NaN,3.0,9.0,NaN,NaN,NaN,NaN,4.0,17.0,2.0,40.0,3.0,G,6.0,"(3, 1)","GA,GAC"
9,20,1234567,microsat1,G,GAC,50.0,"(PASS,)","(0, 1)",NaN,"(0, 2)",NaN,"(1, 1)",NaN,3.0,9.0,NaN,NaN,NaN,NaN,4.0,17.0,2.0,40.0,3.0,G,6.0,"(3, 1)","GA,GAC"


In [ ]:
sample_vcf_df['REF_length'] = sample_vcf_df['REF'].apply(len)
sample_vcf_df['ALT_length'] = sample_vcf_df['ALT'].apply(lambda x: len(x.split(',')))

In [14]:
sample_vcf_df['CHROM']

0    19
1    19
2    20
3    20
4    20
5    20
6    20
7    20
8     X
9     X
Name: CHROM, dtype: object